---

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


In [9]:
# Define the pydantic class with JSON schema for hate speech detection
class HateSpeechAnalysis(BaseModel):
    classification: str = Field(
        ..., description="Classification of the message: 'normal' or 'hate speech'"
    )
    confidence: float = Field(
        ..., ge=0, le=1, description="Confidence score of the classification (0 to 1)"
    )
    reasoning: str = Field(..., description="Brief explanation for the classification")

    class Config:
        schema_extra = {
            "example": {
                "classification": "normal",
                "confidence": 0.95,
                "reasoning": "The message contains no offensive language or discriminatory content.",
            }
        }


In [10]:
# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-2024-05-13",
    temperature=0,
)


In [11]:
# Create a prompt template for hate speech detection
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are an AI trained to detect hate speech. Analyze the given message and determine if it's normal or hate speech. Provide a classification, confidence score, and brief reasoning.",
    ),
    ("human", "Analyze the following message for hate speech: {message}"),
])


In [12]:
# Create the chain with structured output
chain = prompt | llm.with_structured_output(HateSpeechAnalysis)


In [13]:
# Example messages to test
messages = [
    "I love spending time with my family and friends.",
    "All people from that country are criminals and should be deported!",
    "The weather is beautiful today.",
]

for message in messages:
    result = chain.invoke({"message": message})
    print(f"\nAnalysis for: '{message}'")
    print(f"Classification: {result.classification}")
    print(f"Confidence: {result.confidence:.2f}")
    print(f"Reasoning: {result.reasoning}")


Analysis for: 'I love spending time with my family and friends.'
Classification: normal
Confidence: 1.00
Reasoning: The message expresses positive sentiments about spending time with family and friends and contains no harmful or offensive language.

Analysis for: 'All people from that country are criminals and should be deported!'
Classification: hate speech
Confidence: 0.95
Reasoning: The message makes a sweeping generalization about people from a specific country, labeling them all as criminals. It also calls for their deportation, which is a harmful and discriminatory action. Such statements promote xenophobia and discrimination.

Analysis for: 'The weather is beautiful today.'
Classification: normal
Confidence: 1.00
Reasoning: The message is a neutral statement about the weather and does not contain any language that could be considered hateful or offensive.


# now let's use this configuration to process the messages in the processed hateXplain dataset